## 実践演習14-3

scikit-learnのLabelPropagationでirisデータの半教師あり学習を行います。

ライブラリの読み込み

In [0]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.semi_supervised import LabelPropagation

irisデータの読み込み

In [0]:
iris = load_iris()
X = iris.data
y = iris.target

### 半教師ありデータの作成

まず、どのデータの教師信号を消すかを決めます。
[np.random.choice](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html)を使って、データのインデックスを表すndarrayから、指定された個数のインデックスを、重複なしで抜き出します。

In [3]:
unlabeled_points = np.random.choice(np.arange(y.size), int((y.size)*.7), replace=False)
unlabeled_points

array([110,  82,   1,   5, 142,  61,  11, 113,  98,  69, 149, 117,  77,
       137,  25,  94, 119,  48,  40, 120,  51, 102, 103,  54, 115,   0,
        47,  80,  89,  17,  88, 139,  67, 128,  72, 134,  90,  71,  91,
        56, 131,  58, 108,  50,  20, 135,  73,  70,  13,   4,  31,  28,
        32,  35,  21,  24, 126,  22,  10,  59,   3,  68, 118, 130,  41,
        52,  18, 127, 147,  64,  66,  15, 132,  46,  14, 106, 145, 125,
       143,  75,  27,  43,  85,   8,   2,  44, 101,  26,  74, 124,  36,
        81,  49, 105, 114,  65,  23, 133,  95,  83,  33,  39,  34,  87,
       121])

教師ベクトルyを[np.copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.copy.html)を使って変数labelsにコピーし（代入文でコピーすると、オブジェクトが共有されます）、先ほど抜き出したインデックスに対応する値を-1に書き換えます。

In [4]:
labels = np.copy(y)
labels[unlabeled_points] = -1
labels

array([-1, -1, -1, -1, -1, -1,  0,  0, -1,  0, -1, -1,  0, -1, -1, -1,  0,
       -1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1,
       -1, -1, -1,  0,  0, -1, -1, -1,  0, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1,  1, -1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1, -1, -1,  1,
       -1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1, -1,  1,  2, -1,
       -1, -1,  2, -1, -1,  2, -1,  2, -1,  2,  2, -1, -1, -1,  2, -1, -1,
       -1, -1, -1,  2,  2, -1, -1, -1, -1, -1,  2, -1, -1, -1, -1, -1, -1,
        2, -1,  2, -1,  2,  2, -1, -1,  2, -1,  2, -1,  2, -1])

これを教師ベクトルとして、[LabelPropagation](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelPropagation.html)で半教師あり学習を行います。

In [14]:
lp = LabelPropagation(max_iter=10000)
lp.fit(X, labels)

LabelPropagation(gamma=20, kernel='rbf', max_iter=10000, n_jobs=None,
                 n_neighbors=7, tol=0.001)

正解なしデータの割り当て結果を表示します。

In [15]:
lp.score(X[unlabeled_points], y[unlabeled_points])

0.9619047619047619

正解付きデータを5%, 10%, 20%, 30%と変えて、それぞれ100回ずつ学習を試みて性能を評価します。

In [16]:
labeled_percent = [0.05, 0.1, 0.2, 0.3]
num = y.size
for labeled in labeled_percent :
    score = 0
    for i in range(100):
        unlabeled_points = np.random.choice(np.arange(num),int(num-num*labeled), replace=False)
        labels = np.copy(y)
        labels[unlabeled_points] = -1
        lp.fit(X, labels)
        score += lp.score(X[unlabeled_points], y[unlabeled_points])
    print("{0}{1:4.1f}{2}{3:6.3f}".format("labeled:", labeled*100, "%, score=", score/100))        

labeled: 5.0%, score= 0.846
labeled:10.0%, score= 0.926
labeled:20.0%, score= 0.950
labeled:30.0%, score= 0.958
